In [8]:
!pip install pyngrok==4.1.1
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!ngrok authtoken "2FUdUqwnlVWxQc3tzosrwVSXBvq_2tz3tyrU6HdcuUh681bGL" #Without "" marks 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import json
import random
from flask import Flask, render_template, request, session
from flask_ngrok import run_with_ngrok

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    # print(sentence_words)
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for i,w in enumerate(words):
        if w in sentence_words: 
            # assign 1 if current word is in the vocabulary position
            bag[i] = 1
            if show_details:
                print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    # print("p length: " + str(len(p)))
    # print("---------------------------------------------")
    res = model.predict(np.array([p]))[0]
    # print("res: " + str(res))
    ERROR_THRESHOLD = 0.25
    results = []

    for i, r in enumerate(res):
      # print(i, r)
      # print("---------------------")
      if r > ERROR_THRESHOLD:
        results.append([i, r])

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    # print("results: " + str(results))
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    # print(classes)
    print(return_list)
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            print(f"random({i['responses']}) -> {result}")
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [12]:
model = load_model('/content/drive/MyDrive/chatbot-python-project-data-codes/chatbot_model.h5')
intents = json.loads(open('/content/drive/MyDrive/chatbot-python-project-data-codes/intents.json').read())
words = pickle.load(open('/content/drive/MyDrive/chatbot-python-project-data-codes/words.pkl','rb'))
classes = pickle.load(open('/content/drive/MyDrive/chatbot-python-project-data-codes/classes.pkl','rb'))

lemmatizer = WordNetLemmatizer()

In [13]:
app = Flask(__name__, template_folder='/content/drive/MyDrive/chatbot-python-project-data-codes/template')
run_with_ngrok(app)
app.secret_key = "my_secret_key"

@app.route('/')
def index_view():
	session["chatlog"] = ""
	# session.pop("chatlog", None)
	return render_template('index.html')

@app.route('/', methods = ['POST'])
def result():
	if request.method == 'POST':
		to_predict_list = request.form.to_dict()
		msg = to_predict_list["request"]
		response = chatbot_response(msg)
		results = "You: " + msg + "\n" + "Bot: " + response + "\n"
		session["chatlog"] += results	
		return render_template("index.html", result = session["chatlog"])

In [15]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9d43-35-237-80-185.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [08/Oct/2022 01:43:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Oct/2022 01:43:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Oct/2022 01:43:17] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Oct/2022 01:43:30] "POST / HTTP/1.1" 200 -


[{'intent': 'greeting', 'probability': '0.9997873'}]
random(['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?']) -> Hello, thanks for asking


INFO:werkzeug:127.0.0.1 - - [08/Oct/2022 01:44:11] "POST / HTTP/1.1" 200 -


[{'intent': 'options', 'probability': '0.99999917'}]
random(['I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies', 'Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies']) -> Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies


INFO:werkzeug:127.0.0.1 - - [08/Oct/2022 01:44:42] "POST / HTTP/1.1" 200 -


[{'intent': 'goodbye', 'probability': '0.99985373'}]
random(['See you!', 'Have a nice day', 'Bye! Come back again soon.']) -> See you!


INFO:werkzeug:127.0.0.1 - - [08/Oct/2022 01:52:17] "GET / HTTP/1.1" 200 -
